# Ekerå–Håstad's algorithm for computing short discrete logarithms
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Ekerå–Håstads algorithm for computing short discrete logarithms [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20), with improvements from [[E20]](https://doi.org/10.1007/s10623-020-00783-2) and [[E23p]](https://doi.org/10.48550/arXiv.2309.01754), and the post-processing in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).

To start off, let us define the MODP-3072 safe-prime group used in IKEv2, see [RFC 3526](https://datatracker.ietf.org/doc/html/rfc3526) and Tab. 25 on p. 133 in App. A of [NIST SP 800-56A Rev. 3](https://doi.org/10.6028/NIST.SP.800-56Ar3).

In [1]:
from quaspy.math.groups import IntegerModRingMulSubgroupElement;

# Define the safe prime.
p = 5809605995369958062791915965639201402176612226902900533702900882779736177890990861472094774477339581147373410185646378328043729800750470098210924487866935059164371588168047540943981644516632755067501626434556398193186628990071248660819361205119793693985433297036118232914410171876807536457391277857011849897410207519105333355801121109356897459426271845471397952675959440793493071628394122780510124618488232602464649876850458861245784240929258426287699705312584509625419513463605155428017165714465363094021609290561084025893662561222573202082865797821865270991145082200656978177192827024538990239969175546190770645685893438011714430426409338676314743571154537142031573004276428701433036381801705308659830751190352946025482059931306571004727362479688415574702596946457770284148435989129632853918392117997472632693078113129886487399347796982772784615865232621289656944284216824611318709764535152507354116344703769998514148343807;

# Define the generator.
g = IntegerModRingMulSubgroupElement(2, p);

# Define the order of the generator.
r = (p - 1) // 2;

For this group, the exponent $d$ is selected uniformly at random from $[0, 2^{256}) \cap \mathbb Z$.

To this end, we use the [<code>sample_integer(B)</code>](../docs/math/random/sample_integer.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [2]:
from quaspy.math.random import sample_integer;

# Sample d.
m = 256;
d = sample_integer(2 ** m);

print("Sampled d =", d);

# Compute x.
x = g ** d;

print("\nComputed x =", x);

Sampled d = 39445834421792263746237187678251780871950636679959961948118792881895030241537

Computed x = 25730937052929409329300821604826088011445457205584452936587199172592577384458432514514630289588227722878292039568210007992866819369209817597836089186345609640631858782877790821978659248582636472397212975177324748600395293213671286782639966805068753026649317824933051936912513624223796013872563118702295140045377274457838503027149660931604222393683920408480317582687503095087585016121285091684465662591777427232859293714767471971987885599747259655857444946185398794054442119027356347265153659597403651491042205108781731216570009410289484625045666650272960881278468395444085096085191367774828327437143010452266945041606294178430021960369907964053404987058250161979290592509929383840268675260721297520425834372879181723669690313105055830934121103395904931274126637573792731488382133169207327266631935038359990942718404360975704793021398487623800636450053856014578159092069379071501932771648790607297

## 1. Simulating the quantum part of Ekerå–Håstad's algorithm
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau, ..)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) for simulating the quantum part of Ekerå–Håstad's algorithm exactly (up to arbitrary precision) for a given logarithm $d$, order $r$, and parameters $m$, $\ell$ and $\tau$, where $\tau$ is related to the sampling as explained in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).

Below, we use this function to simulate running the algorithm for $d$ and $r$ with control registers of length $m + \ell$ bits and $\ell$ bits respectively.

This yields a frequency pair $(j, k)$ sampled from the probability distribution induced by the algorithm:

In [3]:
from quaspy.logarithmfinding.short.sampling import sample_j_k_given_d_r_tau;

l = m;
tau = 27;

[j, k] = sample_j_k_given_d_r_tau(d, r, m, l, tau);

print("Sampled j =", j);
print("Sampled k =", k);

Sampled j = 63754213727702222666883247198583548998355626313743155351243894725606284027105701604734047720082414957777446139179437453027646826267342071542991574566367
Sampled k = 66280279023355946965346584704632737288343734748410179600493537242010583098246


### 1.1. Solving the frequency pair $(j, k)$ for $d$
We now proceed to solve the frequency pair $(j, k)$ for the logarithm $d$.

To this end, we use the [<code>solve_j_k_for_d(j, k, m, l, g, x, tau, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_j_k_for_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $(j, k)$ for $d$ using the lattice-based post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).

In [4]:
from quaspy.logarithmfinding.short.postprocessing import solve_j_k_for_d;

for [tau, t] in [[4, 2], [7, 2], [14, 2], [17, 2], [27, 2]]:
  recovered_d = solve_j_k_for_d(j, k, m, l, g, x, tau = tau, t = t);
  if recovered_d != None:
    break;

if (recovered_d == d):
  print("Recovered d =", d);

  print("\n[ OK ] Successfully recovered d.");
else:
  print("[FAIL] Failed to recover d.");

Recovered d = 39445834421792263746237187678251780871950636679959961948118792881895030241537

[ OK ] Successfully recovered d.
